In [1]:
#import libraries
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import time
import cv2
import cvzone # provides additional functionalities and tools for computer vision tasks and image processing.
import math
import random



In [3]:
# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)# set the capture dimensions to 1280x720 pixels 


True

In [ ]:
# Hand Detector
detector = HandDetector(detectionCon=0.5, maxHands=1)#The HandDetector class is designed to detect and 
#track hands in a video stream or image. 

In [ ]:
# Find Function
#Each pair of values (x, y) represents a coordinate point on the plane.
x = [300, 245, 200, 170, 145, 130, 112, 103, 93, 87, 80, 75, 70, 67, 62, 59, 57]
y = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
coff = np.polyfit(x, y, 2)  # y = Ax^2 + Bx + C 
print(coff)
# Game Variables
cx, cy = 250, 250
color = (255, 0, 255)
counter = 0 #This variable is  keep track of a specific event or condition.
score = 0
timeStart = time.time()
totalTime = 20

In [ ]:
# Loop
while True:
    success, img = cap.read()
    #This loop continuously captures frames 
    #from the webcam using cap.read() and assigns the result to the img variable.
    img = cv2.flip(img, 1)        #horizontally flip the frame

    if time.time()-timeStart < totalTime:

        hands = detector.findHands(img, draw=False)
        #The detector.findHands() to detect hands in the current frame. 
        if hands:
            lmList = hands[0]['lmList']
            x, y, w, h = hands[0]['bbox']
            x1, y1,_= lmList[5]
            x2, y2,_= lmList[17]
            #The variables x1, y1, x2, and y2 store the 
            #coordinates of specific landmarks (5 and 17).

            distance = int(math.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2))
            A, B, C = coff
            distanceCM = A * distance ** 2 + B * distance + C
            # print(distanceCM, distance)  x=distance
            #It then uses the coefficients from the coff variable (obtained
            #from the previous polynomial fit) to convert the distance to centimeters.

            if distanceCM < 40:
                if x < cx < x + w and y < cy < y + h:
                    counter = 1  # and the center point (cx, cy) is within the 
                    #bounding box of the hand, the counter variable is set to 1.
            
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 3)
            cvzone.putTextRect(img, f'{int(distanceCM)} cm', (x + 5, y - 10))
                #These lines draw a rectangle around the hand's bounding box 
                #and display the calculated distance in centimeters above the rectangle 
        if counter:
            counter += 1
            color = (0, 255, 0)
            if counter == 3: 
                cx = random.randint(100, 1100)
                cy = random.randint(100, 600)#a new random center point (cx, cy) is generated`
                color = (255, 0, 255)
                score +=1
                counter = 0
                 # Draw Button
        cv2.circle(img, (cx, cy), 30, color, cv2.FILLED)
        cv2.circle(img, (cx, cy), 10, (255, 255, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), 20, (255, 255, 255), 2)
        cv2.circle(img, (cx, cy), 30, (50, 50, 50), 2)

        # Game HUD
        cvzone.putTextRect(img, f'Time: {int(totalTime-(time.time()-timeStart))}',
                           (1000, 75), scale=3, offset=20)
        cvzone.putTextRect(img, f'Score: {str(score).zfill(2)}', (60, 75), scale=3, offset=20)
    else:
        cvzone.putTextRect(img, 'Game Over', (400, 400), scale=5, offset=30, thickness=7)
        cvzone.putTextRect(img, f'Your Score: {score}', (450, 500), scale=3, offset=20)
        cvzone.putTextRect(img, 'Press R to restart', (460, 575), scale=2, offset=10)



    cv2.imshow("Image", img)
    key = cv2.waitKey(1)

    if key == ord('r'):
        timeStart = time.time()
        score = 0
